# Prédiction des Allocations d'Actifs - Structure de Recherche Complète

Ce notebook implémente une structure de recherche complète pour prédire le signe des rendements futurs des allocations d'actifs.

**Métrique**: Accuracy (capacité à prédire correctement le signe du rendement futur)

**Objectif**: Pour chaque allocation, prédire si elle devrait être suivie (positive) ou shorée (négative)

## 1. Chargement et Exploration des Données

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import KFold, TimeSeriesSplit, cross_val_score
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import lightgbm as lgbm
from optuna import create_study, trial
import optuna

# Config
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Toutes les librairies importées avec succès")

In [ ]:
# Chargement des données
print("Chargement des données...")
X_train = pd.read_csv('X_train_9xQjqvZ.csv', index_col='ROW_ID')
y_train = pd.read_csv('y_train_Ppwhaz8.csv', index_col='ROW_ID')
X_test = pd.read_csv('X_test_1zTtEnD.csv', index_col='ROW_ID')
sample_submission = pd.read_csv('sample_submission_SpGVFuH.csv', index_col='ROW_ID')

print(f"\n📊 Dimensions des données:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"sample_submission: {sample_submission.shape}")

print(f"\n📋 Colonnes de X_train:")
print(X_train.columns.tolist())

In [ ]:
# Exploration des données
print("\n🔍 Info sur X_train:")
print(f"Types de données:\n{X_train.dtypes.value_counts()}")
print(f"\nValeurs manquantes:\n{X_train.isnull().sum().sum()} valeurs manquantes au total")

print("\n📊 Statistiques descriptives (premiers 5 colonnes):")
print(X_train.iloc[:, :5].describe().T)

print("\n🎯 Distribution de la cible y_train:")
print(y_train['target'].describe())
print(f"\nSigne de la cible:")
print(f"Positif (>0): {(y_train['target'] > 0).sum()} ({(y_train['target'] > 0).sum() / len(y_train) * 100:.1f}%)")
print(f"Négatif (<=0): {(y_train['target'] <= 0).sum()} ({(y_train['target'] <= 0).sum() / len(y_train) * 100:.1f}%)")

# Visualisation de la distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].hist(y_train['target'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Rendement futur')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution des rendements futurs')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)

axes[1].bar(['Positif', 'Négatif'], [(y_train['target'] > 0).sum(), (y_train['target'] <= 0).sum()])
axes[1].set_ylabel('Nombre d\'observations')
axes[1].set_title('Balance des classes')
plt.tight_layout()
plt.show()

## 2. Préparation et Nettoyage des Données

In [ ]:
# Fusion des données
train_data = X_train.copy()
train_data['target'] = y_train['target'].values

print(f"✓ Fusion complète: {train_data.shape}")

# Traitement des valeurs manquantes
print(f"\nValeurs manquantes par colonne (non-zéro):")
missing = train_data.isnull().sum()
print(missing[missing > 0])

# Remplir les valeurs manquantes avec 0 (stratégie acceptable pour ce type de données)
train_data = train_data.fillna(0)
X_test = X_test.fillna(0)

print("✓ Valeurs manquantes remplies")

# Détecter et gérer les outliers
print(f"\nDétection des outliers (± 3 écarts-types):")
base_features = [col for col in train_data.columns if col.startswith('RET_') or col.startswith('SIGNED_VOLUME_')]
outliers_count = 0
for col in base_features[:5]:  # Vérifier les 5 premiers
    Q1 = train_data[col].quantile(0.25)
    Q3 = train_data[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((train_data[col] < Q1 - 3*IQR) | (train_data[col] > Q3 + 3*IQR)).sum()
    if outliers > 0:
        print(f"{col}: {outliers} outliers")

print("\n✓ Données préparées et nettoyées")

## 3. Ingénierie des Features

In [ ]:
# Définir les colonnes de base
RET_features = [f'RET_{i}' for i in range(1, 21)]
SIGNED_VOLUME_features = [f'SIGNED_VOLUME_{i}' for i in range(1, 21)]

print("📈 Construction des features additionnelles...\n")

# === GROUPE 1: Moyennes des rendements à différents horizons ===
for horizon in [3, 5, 10, 15, 20]:
    train_data[f'AVERAGE_PERF_{horizon}'] = train_data[RET_features[:horizon]].mean(1)
    X_test[f'AVERAGE_PERF_{horizon}'] = X_test[RET_features[:horizon]].mean(1)
print("✓ Moyennes des rendements (horizons: 3, 5, 10, 15, 20)")

# === GROUPE 2: Moyennes par allocation et par date ===
for horizon in [3, 5, 10, 15, 20]:
    train_data[f'ALLOCATIONS_AVERAGE_PERF_{horizon}'] = train_data.groupby('TS')[f'AVERAGE_PERF_{horizon}'].transform('mean')
    X_test[f'ALLOCATIONS_AVERAGE_PERF_{horizon}'] = X_test.groupby('TS')[f'AVERAGE_PERF_{horizon}'].transform('mean')
print("✓ Moyennes des rendements agrégées par date")

# === GROUPE 3: Volatilité ===
train_data['STD_PERF_20'] = train_data[RET_features].std(1)
train_data['ALLOCATIONS_STD_PERF_20'] = train_data.groupby('TS')['STD_PERF_20'].transform('mean')
X_test['STD_PERF_20'] = X_test[RET_features].std(1)
X_test['ALLOCATIONS_STD_PERF_20'] = X_test.groupby('TS')['STD_PERF_20'].transform('mean')
print("✓ Volatilité des rendements")

# === GROUPE 4: Statistiques additionnelles des rendements ===
train_data['SKEW_PERF_20'] = train_data[RET_features].skew(1)
train_data['KURT_PERF_20'] = train_data[RET_features].kurtosis(1)
train_data['MIN_PERF_20'] = train_data[RET_features].min(1)
train_data['MAX_PERF_20'] = train_data[RET_features].max(1)
train_data['RANGE_PERF_20'] = train_data['MAX_PERF_20'] - train_data['MIN_PERF_20']

X_test['SKEW_PERF_20'] = X_test[RET_features].skew(1)
X_test['KURT_PERF_20'] = X_test[RET_features].kurtosis(1)
X_test['MIN_PERF_20'] = X_test[RET_features].min(1)
X_test['MAX_PERF_20'] = X_test[RET_features].max(1)
X_test['RANGE_PERF_20'] = X_test['MAX_PERF_20'] - X_test['MIN_PERF_20']
print("✓ Skewness, Kurtosis, Min, Max, Range")

# === GROUPE 5: Features de momentum ===
train_data['RECENT_PERF_5'] = train_data[RET_features[:5]].mean(1)
train_data['EARLY_PERF_5'] = train_data[RET_features[-5:]].mean(1)
train_data['MOMENTUM_RATIO'] = train_data['RECENT_PERF_5'] / (train_data['EARLY_PERF_5'] + 1e-8)

X_test['RECENT_PERF_5'] = X_test[RET_features[:5]].mean(1)
X_test['EARLY_PERF_5'] = X_test[RET_features[-5:]].mean(1)
X_test['MOMENTUM_RATIO'] = X_test['RECENT_PERF_5'] / (X_test['EARLY_PERF_5'] + 1e-8)
print("✓ Features de momentum")

# === GROUPE 6: Features de volume ===
train_data['AVERAGE_VOLUME_20'] = train_data[SIGNED_VOLUME_features].mean(1)
train_data['STD_VOLUME_20'] = train_data[SIGNED_VOLUME_features].std(1)
train_data['RECENT_VOLUME_5'] = train_data[SIGNED_VOLUME_features[:5]].mean(1)

X_test['AVERAGE_VOLUME_20'] = X_test[SIGNED_VOLUME_features].mean(1)
X_test['STD_VOLUME_20'] = X_test[SIGNED_VOLUME_features].std(1)
X_test['RECENT_VOLUME_5'] = X_test[SIGNED_VOLUME_features[:5]].mean(1)
print("✓ Features de volume")

# === GROUPE 7: Interaction turnover-performance ===
train_data['TURNOVER_PERF_INTERACTION'] = train_data['MEDIAN_DAILY_TURNOVER'] * train_data['AVERAGE_PERF_20']
X_test['TURNOVER_PERF_INTERACTION'] = X_test['MEDIAN_DAILY_TURNOVER'] * X_test['AVERAGE_PERF_20']
print("✓ Interactions turnover-performance")

# === GROUPE 8: Features agrégées par groupe ===
train_data['GROUP_AVERAGE_PERF'] = train_data.groupby('GROUP')['AVERAGE_PERF_20'].transform('mean')
train_data['GROUP_AVERAGE_TURNOVER'] = train_data.groupby('GROUP')['MEDIAN_DAILY_TURNOVER'].transform('mean')
X_test['GROUP_AVERAGE_PERF'] = X_test.groupby('GROUP')['AVERAGE_PERF_20'].transform('mean')
X_test['GROUP_AVERAGE_TURNOVER'] = X_test.groupby('GROUP')['MEDIAN_DAILY_TURNOVER'].transform('mean')
print("✓ Features agrégées par groupe")

print(f"\n✓ Feature engineering complet!")
print(f"X_train nouvelles dimensions: {train_data.shape}")
print(f"X_test nouvelles dimensions: {X_test.shape}")

In [ ]:
# Lister toutes les features disponibles
all_features = RET_features + SIGNED_VOLUME_features + ['MEDIAN_DAILY_TURNOVER']
all_features += [f'AVERAGE_PERF_{i}' for i in [3, 5, 10, 15, 20]]
all_features += [f'ALLOCATIONS_AVERAGE_PERF_{i}' for i in [3, 5, 10, 15, 20]]
all_features += ['STD_PERF_20', 'ALLOCATIONS_STD_PERF_20']
all_features += ['SKEW_PERF_20', 'KURT_PERF_20', 'MIN_PERF_20', 'MAX_PERF_20', 'RANGE_PERF_20']
all_features += ['RECENT_PERF_5', 'EARLY_PERF_5', 'MOMENTUM_RATIO']
all_features += ['AVERAGE_VOLUME_20', 'STD_VOLUME_20', 'RECENT_VOLUME_5']
all_features += ['TURNOVER_PERF_INTERACTION']
all_features += ['GROUP_AVERAGE_PERF', 'GROUP_AVERAGE_TURNOVER']

print(f"📊 Total features disponibles: {len(all_features)}")
print(f"   - Base features: {len(RET_features + SIGNED_VOLUME_features + ['MEDIAN_DAILY_TURNOVER'])}")
print(f"   - Features dérivées: {len(all_features) - len(RET_features + SIGNED_VOLUME_features + ['MEDIAN_DAILY_TURNOVER'])}")

## 4. Analyse Exploratoire des Features

In [ ]:
# Analyse des corrélations avec la cible
print("📊 Top 20 features les plus corrélées avec la cible:\n")
correlations = train_data[all_features + ['target']].corr()['target'].drop('target').abs().sort_values(ascending=False)
print(correlations.head(20))

# Visualisation des corrélations
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Top corrélations
top_corr = train_data[correlations.head(15).index.tolist() + ['target']].corr()['target'].drop('target').sort_values()
top_corr.plot(kind='barh', ax=axes[0], color=['red' if x < 0 else 'green' for x in top_corr])
axes[0].set_xlabel('Corrélation avec la cible')
axes[0].set_title('Top 15 features corrélées avec la cible')
axes[0].axvline(x=0, color='black', linestyle='--', linewidth=0.5)

# Distribution des corrélations
axes[1].hist(correlations.values, bins=50, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Corrélation absolue')
axes[1].set_ylabel('Nombre de features')
axes[1].set_title('Distribution des corrélations avec la cible')
axes[1].axvline(x=correlations.mean(), color='red', linestyle='--', label=f'Moyenne: {correlations.mean():.4f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nCorrélation moyenne avec la cible: {correlations.mean():.4f}")
print(f"Max corrélation: {correlations.max():.4f}")
print(f"Min corrélation: {correlations.min():.4f}")

In [ ]:
# Analyse des features par classe
print("\n📈 Analyse des features par classe (Positif vs Négatif):\n")

positive_mask = train_data['target'] > 0
negative_mask = train_data['target'] <= 0

top_20_features = correlations.head(20).index.tolist()

comparison_df = pd.DataFrame({
    'Feature': top_20_features,
    'Mean_Positive': [train_data.loc[positive_mask, feat].mean() for feat in top_20_features],
    'Mean_Negative': [train_data.loc[negative_mask, feat].mean() for feat in top_20_features],
})

comparison_df['Difference'] = comparison_df['Mean_Positive'] - comparison_df['Mean_Negative']
print(comparison_df.to_string())

# Visualisation
fig, ax = plt.subplots(figsize=(10, 8))
comparison_df.sort_values('Difference', ascending=True).plot(
    x='Feature', 
    y=['Mean_Positive', 'Mean_Negative'], 
    kind='barh',
    ax=ax
)
ax.set_xlabel('Moyenne de la feature')
ax.set_title('Moyennes des top features par classe')
plt.tight_layout()
plt.show()

## 5. Entraînement des Modèles de Base

In [ ]:
# Préparation des données pour modélisation
X = train_data[all_features].fillna(0)
y = (train_data['target'] > 0).astype(int)

print(f"✓ Données préparées:")
print(f"  X: {X.shape}")
print(f"  y: {y.shape}")
print(f"  Classes: {y.value_counts().to_dict()}")

# Split temporel (respecter l'ordre temporel)
train_dates = train_data['TS'].unique()
n_train_dates = int(len(train_dates) * 0.7)
train_date_split = train_dates[n_train_dates]

train_mask = train_data['TS'] < train_date_split
test_mask = train_data['TS'] >= train_date_split

X_train_split = X[train_mask]
y_train_split = y[train_mask]
X_val_split = X[test_mask]
y_val_split = y[test_mask]

print(f"\n📊 Split temporel:")
print(f"  Train: {X_train_split.shape[0]} samples")
print(f"  Val: {X_val_split.shape[0]} samples")

# Dictionnaire pour stocker les résultats
results = {}

In [ ]:
# === MODEL 1: Ridge Regression ===
print("🔵 Entraînement Ridge Regression...")
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_split, y_train_split)

ridge_train_acc = accuracy_score(y_train_split, (ridge.predict(X_train_split) > 0.5).astype(int))
ridge_val_acc = accuracy_score(y_val_split, (ridge.predict(X_val_split) > 0.5).astype(int))

results['Ridge'] = {
    'model': ridge,
    'train_acc': ridge_train_acc,
    'val_acc': ridge_val_acc
}

print(f"  Train Accuracy: {ridge_train_acc:.4f}")
print(f"  Val Accuracy: {ridge_val_acc:.4f}")
print()

In [ ]:
# === MODEL 2: LightGBM (Gradient Boosting) ===
print("🟢 Entraînement LightGBM...")
lgbm_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "num_threads": -1,
    "seed": 42,
    "verbosity": -1,
    "learning_rate": 0.05,
    "max_depth": 5,
    "num_leaves": 31,
    "min_child_samples": 20,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
}

train_data_lgbm = lgbm.Dataset(X_train_split, label=y_train_split)
lgbm_model = lgbm.train(lgbm_params, train_data_lgbm, num_boost_round=300, valid_names=['train'])

lgbm_train_pred = (lgbm_model.predict(X_train_split) > 0.5).astype(int)
lgbm_val_pred = (lgbm_model.predict(X_val_split) > 0.5).astype(int)

lgbm_train_acc = accuracy_score(y_train_split, lgbm_train_pred)
lgbm_val_acc = accuracy_score(y_val_split, lgbm_val_pred)

results['LightGBM'] = {
    'model': lgbm_model,
    'train_acc': lgbm_train_acc,
    'val_acc': lgbm_val_acc
}

print(f"  Train Accuracy: {lgbm_train_acc:.4f}")
print(f"  Val Accuracy: {lgbm_val_acc:.4f}")
print()

In [ ]:
# === MODEL 3: Logistic Regression ===
print("🔴 Entraînement Logistic Regression...")
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_split)
X_val_scaled = scaler.transform(X_val_split)

logistic = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
logistic.fit(X_train_scaled, y_train_split)

logistic_train_acc = accuracy_score(y_train_split, logistic.predict(X_train_scaled))
logistic_val_acc = accuracy_score(y_val_split, logistic.predict(X_val_scaled))

results['LogisticRegression'] = {
    'model': logistic,
    'train_acc': logistic_train_acc,
    'val_acc': logistic_val_acc
}

print(f"  Train Accuracy: {logistic_train_acc:.4f}")
print(f"  Val Accuracy: {logistic_val_acc:.4f}")
print()

In [ ]:
# === MODEL 4: Random Forest ===
print("🟠 Entraînement Random Forest...")
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
rf.fit(X_train_split, y_train_split)

rf_train_acc = accuracy_score(y_train_split, rf.predict(X_train_split))
rf_val_acc = accuracy_score(y_val_split, rf.predict(X_val_split))

results['RandomForest'] = {
    'model': rf,
    'train_acc': rf_train_acc,
    'val_acc': rf_val_acc
}

print(f"  Train Accuracy: {rf_train_acc:.4f}")
print(f"  Val Accuracy: {rf_val_acc:.4f}")
print()

In [ ]:
# Résumé des résultats
print("📊 RÉSUMÉ DES MODÈLES DE BASE:\n")
summary_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Train Accuracy': [results[m]['train_acc'] for m in results.keys()],
    'Val Accuracy': [results[m]['val_acc'] for m in results.keys()]
})
print(summary_df.to_string(index=False))

best_model_name = summary_df.loc[summary_df['Val Accuracy'].idxmax(), 'Model']
best_val_acc = summary_df['Val Accuracy'].max()
print(f"\n✨ Meilleur modèle: {best_model_name} (Accuracy: {best_val_acc:.4f})")

# Visualisation
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(summary_df))
width = 0.35
ax.bar(x - width/2, summary_df['Train Accuracy'], width, label='Train', alpha=0.8)
ax.bar(x + width/2, summary_df['Val Accuracy'], width, label='Val', alpha=0.8)
ax.set_ylabel('Accuracy')
ax.set_title('Comparaison des modèles de base')
ax.set_xticks(x)
ax.set_xticklabels(summary_df['Model'])
ax.legend()
ax.set_ylim([0.5, 0.52])  # Zoom pour voir les différences
plt.tight_layout()
plt.show()

## 6. Cross-Validation avec Time Series Split

In [ ]:
print("🔄 Cross-Validation LightGBM avec Time Series Split (8 splits)...\n")

n_splits = 8
train_dates_cv = train_data['TS'].unique()
cv_scores = []
cv_models = []

splits = KFold(n_splits=n_splits, random_state=42, shuffle=True).split(train_dates_cv)

for i, (train_idx, val_idx) in enumerate(splits):
    train_dates_fold = train_dates_cv[train_idx]
    val_dates_fold = train_dates_cv[val_idx]
    
    fold_train_mask = train_data['TS'].isin(train_dates_fold)
    fold_val_mask = train_data['TS'].isin(val_dates_fold)
    
    X_fold_train = X[fold_train_mask]
    y_fold_train = y[fold_train_mask]
    X_fold_val = X[fold_val_mask]
    y_fold_val = y[fold_val_mask]
    
    # Entraînement
    lgbm_data = lgbm.Dataset(X_fold_train, label=y_fold_train)
    model = lgbm.train(lgbm_params, lgbm_data, num_boost_round=300, valid_names=['train'])
    
    # Évaluation
    y_pred = (model.predict(X_fold_val) > 0.5).astype(int)
    score = accuracy_score(y_fold_val, y_pred)
    cv_scores.append(score)
    cv_models.append(model)
    
    print(f"Fold {i+1:2d}/{n_splits} - Accuracy: {score:.4f}")

mean_score = np.mean(cv_scores)
std_score = np.std(cv_scores)
print(f"\n📊 Résultat CV LightGBM:")
print(f"  Mean Accuracy: {mean_score:.4f}")
print(f"  Std Dev: {std_score:.4f}")
print(f"  Range: [{mean_score - std_score:.4f}, {mean_score + std_score:.4f}]")

In [ ]:
# Visualisation des scores CV
plt.figure(figsize=(10, 5))
plt.plot(range(1, n_splits+1), cv_scores, marker='o', linewidth=2, markersize=8)
plt.axhline(y=mean_score, color='r', linestyle='--', label=f'Mean: {mean_score:.4f}')
plt.fill_between(range(1, n_splits+1), mean_score - std_score, mean_score + std_score, alpha=0.2)
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('Evolution de l\'Accuracy par fold (CV LightGBM)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Analyse de l'Importance des Features

In [ ]:
# Importance des features - Moyenne sur tous les modèles CV
print("📊 Feature Importance (moyenne des 8 folds):\n")

feature_importance_df = pd.DataFrame({
    'Feature': all_features,
    'Importance': np.mean([model.feature_importance(importance_type='gain') for model in cv_models], axis=0)
})

feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
print(feature_importance_df.head(20).to_string(index=False))

# Top 20 features
top_20_importance = feature_importance_df.head(20)
plt.figure(figsize=(10, 8))
plt.barh(range(len(top_20_importance)), top_20_importance['Importance'].values)
plt.yticks(range(len(top_20_importance)), top_20_importance['Feature'].values)
plt.xlabel('Importance (Gain)')
plt.title('Top 20 Features par Importance (LightGBM)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Sauvegarder top 30 features pour éventuellement les utiliser
top_30_features = feature_importance_df.head(30)['Feature'].tolist()
print(f"\n✓ Top 30 features identifiées: {top_30_features}")

## 8. Entraînement du Modèle Final et Prédictions

In [ ]:
# Entraîner le modèle final sur TOUTES les données d'entraînement
print("🚀 Entraînement du modèle final sur toutes les données...\n")

lgbm_final_params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "num_threads": -1,
    "seed": 42,
    "verbosity": -1,
    "learning_rate": 0.05,
    "max_depth": 5,
    "num_leaves": 31,
    "min_child_samples": 20,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
}

train_data_final = lgbm.Dataset(X, label=y)
model_final = lgbm.train(lgbm_final_params, train_data_final, num_boost_round=300, valid_names=['train'])

print("✓ Modèle final entraîné")

# Générer les prédictions sur le test set
print(f"\n📊 Génération des prédictions sur {X_test.shape[0]} samples...\n")

# Prédictions brutes
X_test_features = X_test[all_features].fillna(0)
predictions_raw = model_final.predict(X_test_features)

# Conversion en classes
predictions_binary = (predictions_raw > 0.5).astype(int)

print(f"✓ Prédictions générées")
print(f"  Distribution: {np.bincount(predictions_binary)}")
print(f"  Proportion positif: {predictions_binary.mean():.2%}")

In [ ]:
# Ensemble: Moyenne des prédictions des modèles CV (stacking simple)
print("\n📈 Génération d'un ensemble (ensemble moyenne des CV folds)...\n")

ensemble_predictions = np.zeros(len(X_test_features))
for i, model in enumerate(cv_models):
    pred = model.predict(X_test_features)
    ensemble_predictions += pred
    print(f"  Fold {i+1}: {(pred > 0.5).mean():.2%} positif")

ensemble_predictions /= len(cv_models)
ensemble_binary = (ensemble_predictions > 0.5).astype(int)

print(f"\n✓ Ensemble généré")
print(f"  Distribution: {np.bincount(ensemble_binary)}")
print(f"  Proportion positif: {ensemble_binary.mean():.2%}")

## 9. Génération des Fichiers de Soumission

In [ ]:
# Option 1: Soumission avec le modèle final
submission_final = sample_submission.copy()
submission_final['target'] = predictions_binary
submission_final.to_csv('submission_lgbm_final.csv')
print("✓ submission_lgbm_final.csv sauvegardée")

# Option 2: Soumission avec l'ensemble (meilleure pratique)
submission_ensemble = sample_submission.copy()
submission_ensemble['target'] = ensemble_binary
submission_ensemble.to_csv('submission_ensemble.csv')
print("✓ submission_ensemble.csv sauvegardée")

# Option 3: Soumission avec Ridge (baseline)
ridge_test_pred = (ridge.predict(X_test[all_features].fillna(0)) > 0.5).astype(int)
submission_ridge = sample_submission.copy()
submission_ridge['target'] = ridge_test_pred
submission_ridge.to_csv('submission_ridge_baseline.csv')
print("✓ submission_ridge_baseline.csv sauvegardée")

print(f"\n📋 Vérification des fichiers générés:")
print(f"  submission_lgbm_final.csv: {submission_final.shape}")
print(f"  submission_ensemble.csv: {submission_ensemble.shape}")
print(f"  submission_ridge_baseline.csv: {submission_ridge.shape}")

In [ ]:
# Afficher un aperçu des prédictions
print("\n📊 Aperçu des prédictions:\n")
comparison = pd.DataFrame({
    'Ridge': submission_ridge['target'].values,
    'LightGBM_Final': submission_final['target'].values,
    'Ensemble': submission_ensemble['target'].values
})

print("Premiers 10 exemples:")
print(comparison.head(10).to_string())

print("\nAccord entre modèles:")
print(f"  Ridge vs LightGBM: {(comparison['Ridge'] == comparison['LightGBM_Final']).mean():.2%}")
print(f"  Ridge vs Ensemble: {(comparison['Ridge'] == comparison['Ensemble']).mean():.2%}")
print(f"  LightGBM vs Ensemble: {(comparison['LightGBM_Final'] == comparison['Ensemble']).mean():.2%}")

# Distribution des prédictions
print(f"\nDistribution des prédictions:")
print(f"  Ridge - Positif: {submission_ridge['target'].sum():5d} ({submission_ridge['target'].mean():.2%})")
print(f"  LightGBM - Positif: {submission_final['target'].sum():5d} ({submission_final['target'].mean():.2%})")
print(f"  Ensemble - Positif: {submission_ensemble['target'].sum():5d} ({submission_ensemble['target'].mean():.2%})")

## 10. Résumé Exécutif

In [ ]:
print("=" * 70)
print("RÉSUMÉ EXÉCUTIF - PRÉDICTION ALLOCATIONS D'ACTIFS")
print("=" * 70)

print(f"\n📊 DONNÉES:")
print(f"  - Ensemble d'entraînement: {X.shape[0]:,} samples × {X.shape[1]} features")
print(f"  - Ensemble de test: {X_test_features.shape[0]:,} samples")
print(f"  - Équilibre des classes: {y.mean():.1%} positif / {(1-y.mean()):.1%} négatif")

print(f"\n🔧 FEATURES ENGINEERING:")
print(f"  - Base features: 41 (RET, SIGNED_VOLUME, MEDIAN_DAILY_TURNOVER)")
print(f"  - Features dérivées: {len(all_features) - 41}")
print(f"  - Total features: {len(all_features)}")

print(f"\n🤖 MODÈLES ENTRAÎNÉS:")
print(f"  - Ridge Regression: Val Accuracy = {ridge_val_acc:.4f}")
print(f"  - Logistic Regression: Val Accuracy = {logistic_val_acc:.4f}")
print(f"  - Random Forest: Val Accuracy = {rf_val_acc:.4f}")
print(f"  - LightGBM: Val Accuracy = {lgbm_val_acc:.4f}")

print(f"\n✓ CROSS-VALIDATION LIGHTGBM (8 folds):")
print(f"  - Mean Accuracy: {mean_score:.4f}")
print(f"  - Std Dev: {std_score:.4f}")
print(f"  - Confidence Interval: [{mean_score - std_score:.4f}, {mean_score + std_score:.4f}]")

print(f"\n🎯 TOP 5 FEATURES (par importance):")
for i, (feat, imp) in enumerate(feature_importance_df.head(5).values, 1):
    print(f"  {i}. {feat}: {imp:.2f}")

print(f"\n📈 PRÉDICTIONS GÉNÉRÉES:")
print(f"  - Modèle Final (LightGBM sur all data)")
print(f"    → Positif: {submission_final['target'].sum():,} ({submission_final['target'].mean():.2%})")
print(f"  - Ensemble (moyenne 8 CV folds)")
print(f"    → Positif: {submission_ensemble['target'].sum():,} ({submission_ensemble['target'].mean():.2%})")
print(f"  - Ridge Baseline")
print(f"    → Positif: {submission_ridge['target'].sum():,} ({submission_ridge['target'].mean():.2%})")

print(f"\n💾 FICHIERS GÉNÉRÉS:")
print(f"  ✓ submission_lgbm_final.csv")
print(f"  ✓ submission_ensemble.csv")
print(f"  ✓ submission_ridge_baseline.csv")

print("\n" + "=" * 70)
print("RECOMMANDATION: Soumettre 'submission_ensemble.csv' (moyenne CV)")
print("=" * 70)